# Binäre Klassifikation anhand eines Spam-Filters

## Technische Vorbereitung
Damit alle notwendigen Funktionen und Befehle zur Verfügung stehen, müssen zunächst einige Module mit ihren jeweiligen Klassen importiert werden. Dazu zählt pandas und scikit-learn. Alle Module werden für spätere Datenanalysemethoden genutzt.


In [17]:
import pandas as pd
from pandas import set_option
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV

## Analyse des Datensatzes
Der Datensatz liegt hierbei in einer ".csv" - Datei und beinhaltet 5572 Datensätze. Er ist in zwei Spalten untergliedert. Die Spalte Label beinhaltet die Gruppen Spam und Ham die den jeweiligen Datensatz zugeordnet sind. Bei der zweiten Spalte handelt es sich um den EmailText.


In [26]:
dataframe = pd.read_csv("spam.csv")
print(dataframe.head())
print("")
print(dataframe.describe())


  Label                                          EmailText
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

       Label               EmailText
count   5572                    5572
unique     2                    5169
top      ham  Sorry, I'll call later
freq    4825                      30


Sowohl Label als auch der EmailText besitzen den Datentyp object.

In [28]:
print(dataframe.dtypes)


Label        object
EmailText    object
dtype: object


Durch den folgenden Code wurde die Anzahl von Spam- bzw. Ham-Emails in dem Datensatz gezählt. Dabei fällt auf, dass es sehr unausgeglichen ist und es sich bei den meisten Nachrichten um ham-Emails handelt.

In [84]:
hamCount = dataframe.apply(lambda x: True if x["Label"] == "ham" else False, axis=1)
spamCount = dataframe.apply(lambda x: True if x["Label"] == "spam" else False, axis=1)

print("Anzahl ham  - Emails: ", len(hamCount[hamCount == True].index))
print("Anzahl spam - Emails: ", len(spamCount[spamCount == True].index))


Anzahl ham  - Emails:  4825
Anzahl spam - Emails:  747


## Traings- und Testdaten
Im nächsten Schritt werden die Daten in den Spalten separiert und danach gruppiert. Die Trainingsdaten werden für das Lernen der Muster und Zusammenhänge in den Daten verwendet. Der Algorithmus nutzt diese Daten um daraus zu lernen. Bei den Testdaten handelt es sich um Daten mit der gleichen Wahrscheinlichkeitsverteilung. Diese Daten werden vom Algorithmus vorher nicht genutzt. Mit ihnen kann nachgeweisen werden mit welcher Qualität der Algorithmus auf neue Daten reagieren wird. Die Verteilung verläuft auf 80% Trainingsdaten und 20% Testdaten.

In [90]:
x = dataframe["EmailText"]
y = dataframe["Label"]

x_train,y_train = x[0:4457],y[0:4457]
x_test,y_test = x[4457:],y[4457:]

## Transformieren der Daten
Nachdem die Daten gruppiert wurden müssen die Strings in numerische Werte umgewandelt werden, damit der Klassifikator die Daten nutzen kann. Scikit-Learn stellt hierbei einen "CountVectorizer" zur Verfügung. Er konvertiert eine Sammlung von Text Dokumenten in eine Matrix mit Token-Zahlen

In [89]:
cv = CountVectorizer()  
features = cv.fit_transform(x_train)

## Klassifikator
Für den Spam-Filter wird der Support-Vector-Machine-Klassifikator genutzt. Damit die einzelnen Klassen einen möglichst breiten Abstand ohne Objekte zueinander besitzen. Des Weiteren müssen auch die Testdaten transformiert werden. 

In [13]:
model = svm.SVC()
model.fit(features,y_train)

features_test = cv.transform(x_test)
print("Die Genauigkeit des Models entspricht:", model.score(features_test,y_test))

Die Genauigkeit des Models entspricht: 0.9856502242152466


In [91]:
tuned_parameters = {'kernel': ['rbf','linear'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}

model = GridSearchCV(svm.SVC(), tuned_parameters)

model.fit(features,y_train)

print(model.best_params_)

features_test = cv.transform(x_test)
print("Die Genauigkeit des Models entspricht:", model.score(features_test,y_test))

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Die Genauigkeit des Models entspricht: 0.9874439461883409
